In [0]:
# This file trains the USTC using differential privacy 
# by adding nosie to autoencoder first layer 
# It uses a dense autoencoder. This file enables training 
# with both l1, l2 noise and both laplacian and gaussian 
# noise. See the https://arxiv.org/pdf/1802.03471.pdf for 
# details about how this works.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Move to drive
%cd drive
%cd 'My Drive'
%cd Oxford-Thesis

In [0]:
import pickle
import numpy as np
import numpy as np
import tensorflow as tf
import six 
import math
from tensorflow.python.training import moving_averages 
import pickle
import tensorflow.keras.backend as K
from sklearn.preprocessing import normalize
import tensorflow_probability as tfp
CONT_FEATURE_SIZE = 38
NUM_BENIGN = 187426
NUM_SCAN = 2073
NUM_FUZZ = 1087
NUM_ANALY = 558
NUM_BACK = 92
NUM_DOS = 1709
NUM_EXP = 11481
NUM_GEN = 1699
NUM_SHELL = 0
NUM_WORM = 148

In [0]:
## Constants for deciding how to add differential privacy to autoencoder 
NOISE_PLACEMENT = 'img_noise'
NOISE_SCHEME = 'l1_l1'
FILTER_SIZES = 1
NUM_FILTERS = 1 
STRIDES = 1 
EPSILON = 1.0
DELTA_DP = 0.05
SCALE = 0.1
NOISE = 0
BATCH_SHAPE = 128
NOISE_TYPE = 'laplace'
AUTO_ENCODER_OUT_SHAPE = 158
CONT_FEATURE_SIZE = 158
FOLDER = 'UNSW-AutoEncoderLaplace'
NUM_CLASS =10
TRAIN =True 
SCALE_STRING = "0point1"

In [0]:
## Load in data 

## Load in malicious training data
with open('./embedding/CNN_FULL/UNSW_mal_meta_train', 'rb') as fp:
  mal_train_meta_X = pickle.load(fp)
  fp.close()
  
## Load in malicious validation data
with open('./embedding/CNN_FULL/UNSW_mal_meta_valid', 'rb') as fp:
  mal_valid_meta_X = pickle.load(fp)
  fp.close()
  
## Load in benign traning data  
with open('./embedding/CNN_FULL/UNSW_train_benign_meta', 'rb') as fp:
  benign_train_meta_X =pickle.load(fp)
  fp.close()
  
## Load in benign valid data
with open('./embedding/CNN_FULL/UNSW_valid_benign_meta', 'rb') as fp:
  benign_valid_meta_X = pickle.load(fp)
  fp.close()

In [0]:
train_labels = [[0,1]]*len(benign_train_meta_X) +[[1,0]]*len(mal_train_meta_X)
valid_labels = [[0,1]]*len(benign_valid_meta_X) +[[1,0]]*len(mal_valid_meta_X)

In [0]:
## Randomize training and validation data 
train_X = np.concatenate([benign_train_meta_X,mal_train_meta_X])
print(train_X.shape)
c = list(zip(list(train_X),train_labels))
np.random.shuffle(c)
train_X, train_labels = zip(*c)
valid_X = np.concatenate([benign_valid_meta_X,mal_valid_meta_X])
c = list(zip(list(valid_X),valid_labels))
np.random.shuffle(c)
valid_X, valid_labels = zip(*c)

train_X = np.array(train_X)
valid_X = np.array(valid_X)
train_X = train_X.astype(float)
valid_X = valid_X.astype(float)

In [0]:
## DP-Mult for different types of noise 
def dp_mult( size=None):
  epsilon_dp =EPSILON
  delta_dp   = DELTA_DP
  max_pixeldp_norm = SCALE
  if NOISE_SCHEME == 'l1_l2'    or  \
      NOISE_SCHEME == 'l2':
      return max_pixeldp_norm * \
          math.sqrt(2 * math.log(1.25 / delta_dp)) / epsilon_dp
  elif NOISE_SCHEME == 'l1_l1'  or  \
      NOISE_SCHEME == 'l1':
      return max_pixeldp_norm / epsilon_dp

In [0]:

## Create different types of noises for different 
## noise types and schemes
def img_dp_noise(shape):
  if NOISE_TYPE == 'laplace':
    sensitivity_multiplier = EPSILON 
    dp_mult_x = dp_mult()
    loc = tf.zeros([BATCH_SHAPE,shape])
    scale = tf.ones([BATCH_SHAPE,shape])
    epsilon = tfp.distributions.Laplace(loc,scale).sample()
    noise_scale = tf.placeholder(tf.float32,shape =(), name ='noise_scale')
    L1_SENSITIVITY = 1.0
    b = L1_SENSITIVITY *dp_mult_x*SCALE
    noise = b *epsilon
    
  elif NOISE_TYPE == 'gaussian':
    dp_mult_x = dp_mult()
    epsilon = tf.random_normal(shape=(BATCH_SHAPE,shape), mean=0,stddev=1)
    sensitivity_multiplier = EPSILON 
    L2_SENSITIVITY = 1.0
    noise_scale = tf.placeholder(tf.float32,shape =(), name ='noise_scale')
    SIGMA = tf.multiply(dp_mult_x,L2_SENSITIVITY,name= "SIGMA")
    noise_stddev = SCALE * SIGMA
    noise = noise_stddev *epsilon
  return noise

In [0]:
# Normalize the l1 direction https://github.com/columbia/pixeldp
def l1_normalize(x, dim, epsilon=1e-12, name=None):
  """Normalizes along dimension `dim` using an L1 norm.
  For a 1-D tensor with `dim = 0`, computes
      output = x / max(sum(abs(x)), epsilon)
  For `x` with more dimensions, independently normalizes each 1-D slice along
  dimension `dim`.
  Args:
    x: A `Tensor`.
    dim: Dimension along which to normalize.  A scalar or a vector of
      integers.
    epsilon: A lower bound value for the norm. Will use `sqrt(epsilon)` as the
      divisor if `norm < sqrt(epsilon)`.
    name: A name for this operation (optional).
  Returns:
    A `Tensor` with the same shape as `x`.
  """
  with tf.name_scope(name, "l1_normalize", [x]) as name:
    with tf.device("/gpu:0"):
      x          = tf.convert_to_tensor(x, name            = "x")
      abs_sum    = tf.reduce_sum(tf.abs(x), dim, keep_dims = True)
      x_inv_norm = tf.reciprocal(tf.maximum(abs_sum, epsilon))
      return tf.multiply(x, x_inv_norm, name=name)

In [0]:
with tf.device('/device:GPU:0'):
  NOISE = img_dp_noise(int(CONT_FEATURE_SIZE/2))
  ## Define the auto encoder without keras 
  x_ = tf.placeholder(tf.float32,shape= [BATCH_SHAPE,CONT_FEATURE_SIZE])
  y_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,CONT_FEATURE_SIZE])
  
  x2_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,AUTO_ENCODER_OUT_SHAPE])
  y2_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,2])
  y3_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,10])

In [0]:
def get_rep(x, full_rep = True):
  with tf.variable_scope("first_model",reuse=tf.AUTO_REUSE):
    with tf.device('/device:GPU:0'):
    
      ## ADD NOISE 
      #noise_image = tf.add(NOISE,x) 
      W_dense1 = tf.get_variable("dense1",initializer =tf.random_normal(stddev=0.1, shape =[CONT_FEATURE_SIZE,int(CONT_FEATURE_SIZE/2)]))
      b_dense1 =  tf.get_variable("bias1",initializer =tf.random_normal(stddev=0.1, shape =[int(CONT_FEATURE_SIZE/2)]))
      if NOISE_SCHEME == "l1_l2":
        W_dense1_norm = tf.nn.l2_normalize(W_dense1,dim=1)
      else:
        W_dense1_norm = l1_normalize(W_dense1,dim=1)

      h_fc1 = tf.nn.relu(tf.matmul(tf.cast(x,tf.float32), W_dense1_norm))
      if TRAIN == True:
        noise_image = tf.add(h_fc1,NOISE)
      else:
        noise_image = tf.add(h_fc1,img_dp_noise(int(CONT_FEATURE_SIZE/2)))
      
      W_dense2 =tf.get_variable("dense2",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/2),int(CONT_FEATURE_SIZE/4)]))
      b_dense2 = tf.get_variable("bias2",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4)]))
      h_fc2 = tf.nn.relu(tf.matmul(noise_image, W_dense2) + b_dense2)


      W_dense3 =tf.get_variable("dense3",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4),10]))
      b_dense3 =tf.get_variable("bias3",initializer =tf.random_normal(stddev=0.1,shape= [10])) 
      first_output = tf.matmul(h_fc2, W_dense3) + b_dense3
      h_fc3 = tf.nn.relu(first_output)

      W_dense4 =tf.get_variable("dense4",initializer =tf.random_normal(stddev=0.1,shape= [10,int(CONT_FEATURE_SIZE/4)]))
      b_dense4 =tf.get_variable("bias4",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4)]))
      h_fc4 = tf.nn.relu(tf.matmul(h_fc3, W_dense4) + b_dense4)

      W_dense5 = tf.get_variable("dense5",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4),int(CONT_FEATURE_SIZE/2)]))
      b_dense5 = tf.get_variable("bias5",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/2)]))
      h_fc5 = tf.nn.relu(tf.matmul(h_fc4, W_dense5) + b_dense5)

      W_dense6 = tf.get_variable("dense6",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/2),int(CONT_FEATURE_SIZE)]))
      b_dense6 = tf.get_variable("bias6",initializer =tf.random_normal(stddev=0.1,shape= [CONT_FEATURE_SIZE]))
      h_fc6 = tf.matmul(h_fc5, W_dense6) + b_dense6
      if full_rep == True:
        return h_fc6
      return first_output

In [0]:
def get_logits_cnn(x,return_embedding= False):
  with  tf.variable_scope("tree_second_model",reuse=tf.AUTO_REUSE):
    with  tf.device('/device:GPU:0'):
      W_dense4_class = tf.get_variable("dense4_sec",initializer =tf.random_normal(stddev=0.1, shape =[AUTO_ENCODER_OUT_SHAPE,784] ))
      b_dense4_class = tf.get_variable("bias4_sec",initializer =tf.random_normal(stddev=0.1,shape=[784]))
      h_fc4_class = tf.nn.relu(tf.matmul(tf.cast(x,tf.float32), W_dense4_class) + b_dense4_class)

      reshape_1_class = tf.reshape(h_fc4_class,[BATCH_SHAPE,28,28,1])
      conv2d_1_weight = tf.get_variable("conv2d_1_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 1, 32]))
      conv2d_1_bias = tf.get_variable("conv2d_1_bias",initializer =tf.random_normal(stddev=0.1, shape= [32]))
      conv2d_1_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(reshape_1_class,
                                                  conv2d_1_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_1_bias))

      maxpool2d_1_class = tf.nn.max_pool(conv2d_1_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')
      conv2d_2_weight = tf.get_variable("conv2d_2_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 32, 64]))
      conv2d_2_bias = tf.get_variable("conv2d_2_bias",initializer =tf.random_normal(stddev=0.1, shape=[64]))
      conv2d_2_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(maxpool2d_1_class,
                                                  conv2d_2_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_2_bias))

      maxpool2d_2_class = tf.nn.max_pool(conv2d_2_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')

      reshape_2_class = tf.reshape(maxpool2d_2_class,[BATCH_SHAPE,-1])
      W_dense5_class = tf.get_variable("dense5_sec",initializer =tf.random_normal(stddev=0.1,shape=[7*7*64,1024]))
      b_dense5_class = tf.get_variable("bias5_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024]))
      h_fc5_logit = tf.nn.bias_add(tf.matmul(reshape_2_class, W_dense5_class),b_dense5_class)
      h_fc5_class = tf.nn.relu(h_fc5_logit)

      out_weight_class =  tf.get_variable("out_dense_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024, 2]))
      out_bias_class =  tf.get_variable("out_bias_sec",initializer =tf.random_normal(stddev=0.1,shape = [2]))
      out_final_logits = tf.nn.bias_add(tf.matmul(h_fc5_class,out_weight_class),out_bias_class)
      if return_embedding == True:
        return h_fc5_logit
      return out_final_logits

In [0]:
with tf.device('/device:GPU:0'):

  mse = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_, predictions=get_rep(x_,True)))
  train_step = tf.train.AdamOptimizer(1e-4,name="adam-encoder").minimize(mse)
  accuracy = mse

In [0]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
print("GPU Available: ", tf.test.is_gpu_available())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
saver = tf.train.Saver()

In [0]:
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
with session as sess:
  noise = sess.run(NOISE)

In [0]:
## Dump noise for later use and training the CNN later 
with open('./DP-GRAPHS/' + FOLDER +'/NOISE'+SCALE_STRING, 'wb') as fp:
  pickle.dump(noise,fp)
  fp.close()

In [0]:
## Train the autoencoder
EPOCHS = 60
mses = []
with session as sess:
  for epoch in range(EPOCHS):
    for batch_num in range(int(len(train_X)/BATCH_SHAPE)):
      batch = train_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_label = train_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      train_accuracy += accuracy.eval(feed_dict={x_:batch, y_: batch_label})
      train_step.run(feed_dict={x_: batch, y_: batch_label})
    mses.append(train_accuracy/(int(len(train_X)/BATCH_SHAPE)))
    print('Step %d, training accuracy %g' % (batch_num , train_accuracy/(int(len(train_X)/BATCH_SHAPE))))
    valid_acc = 0 
    for batch_num in range(int(len(valid_X)/BATCH_SHAPE)): 
      batch = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_label = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      valid_acc += accuracy.eval(feed_dict={x_:batch, y_: batch_label})
    print('Step %d, validation accuracy %g' % (batch_num , valid_acc/(int(len(valid_X)/BATCH_SHAPE)))) 
  saver.save(sess, './DP-GRAPHS/' + FOLDER+ '/autoencoder'+SCALE_STRING+'.ckpt')